# Reinforcement Tasks

En el problemas de aprendizaje, a diferencia de en programación dinámica, no conocemos los detalles del problema. Por lo tanto vamos a modelar un problema de aprendizaje con dos "clases". 

1. La primera clase va a definir la tarea. Vamos a nombrar a esta tarea RLEnvironment
2. La segunda clase define al agente que aprende a resolver la tarea maximizando su ganancia.

Vamos a ver como construir cada una de estas clases de manera que tengamos un cascarón para distintas tareas de aprendizaje.


## Una tarea de ejemplo: el problema el viajero

Para poder entender como modelar un problema con estas clases. Vamos a programar un agente que aprenda a encontrar un camino optimo para visitar todas las ciudades de Europa empezando desde cualquier ciudad inicial.

La informacion de distancia entre ciudades esta dada en la siguiente tabla

In [1]:
dist_info = readcsv("data/DistMat.csv")

25×25 Array{Any,2}:
 "Origin"                "Barcelona"  …      "Vienna"      "Warsaw"
 "Barcelona"            0                1347.43       1862.33     
 "Belgrade"          1528.13              489.28        826.66     
 "Berlin"            1497.61              523.61        516.06     
 "Brussels"          1062.89              914.81       1159.85     
 "Bucharest"         1968.42          …   855.32        946.12     
 "Budapest"          1498.79              216.98        545.29     
 "Copenhagen"        1757.54              868.87        667.8      
 "Dublin"            1469.29             1680          1823.72     
 "Hamburg"           1471.78              742.79        750.49     
 "Istanbul"          2230.42          …  1273.88       1386.08     
 "Kiev"              2391.06             1052.76        690.12     
 "London"            1137.67             1233.48       1445.85     
 "Madrid"             504.64             1807.09       2288.42     
 "Milan"              725.12

Una de las estrucutras basicas de Julia son los NamedArrays, que funcionan de manera similar a las matrices con nombres de dimensiones en R. Usaremos NamedArrays para guardar la informacion de distancias de manera que sea facil de accesar. Para poder usar NamedArrays necesitamos usar una paqueteria adicional a julia base con el comando:

In [2]:
# Pkg.add("NamedArrays") # para instalar por primera vez
using NamedArrays

A continuacion creamos un named array con la informacion de distancias y nombres dimenciones las ciudades

In [3]:
cities = dist_info[2:end, 1] 
distances = dist_info[2:end, 2:end]
distArray = NamedArray(distances, (cities, cities))

24×24 Named Array{Any,2}
           A ╲ B │        Barcelona  …            Warsaw
─────────────────┼──────────────────────────────────────
Barcelona        │                0  …           1862.33
Belgrade         │          1528.13               826.66
Berlin           │          1497.61               516.06
Brussels         │          1062.89              1159.85
Bucharest        │          1968.42               946.12
Budapest         │          1498.79               545.29
Copenhagen       │          1757.54                667.8
Dublin           │          1469.29              1823.72
Hamburg          │          1471.78               750.49
Istanbul         │          2230.42              1386.08
Kiev             │          2391.06               690.12
London           │          1137.67              1445.85
Madrid           │           504.64              2288.42
Milan            │           725.12              1143.01
Moscow           │          3006.93              1149.41
Munich

In [4]:
d = distArray["Barcelona", "Warsaw"]
println("Distance from Barcelona to Warsaw: $d km")

Distance from Barcelona to Warsaw: 1862.33 km


Hubieramos podido implementar algo similar usando diccionarios, que son un tipo nativo de estructura de datos de julia y muchos otros lenguages. Pongo aqui la implementacion solo por curiosidad, aunque usaremos `distArray` para todos los calculos futuros.

In [5]:
distDict = Dict([([cities[i], cities[j]], distances[i, j]) for i in 1:24, j in 1:24])
d = distDict[["Barcelona", "Warsaw"]] # notemos los brackets adicionales
println("Distance from Barcelona to Warsaw: $d km")

Distance from Barcelona to Warsaw: 1862.33 km


## Reinforcement Learning Environments

Los elementos que debe tener una tarea de aprendizaje son:

1. Un espacio de estados
2. Un espacio de acciones disponibles
3. Un funcion de transicion

In [6]:
type RLEnv # reinforcement learning environment, should be immutable for efficiency
    state_space::Array{Any, 1} # 
    trans_fun::Function # (state, action) -> (new_state, reward)
    action_set::Function # (state) -> (array of available actions from state_space)
end

Por ejemplo, en nuestro problema el espacio de estados y de acciones es el mismo. Se puede estar en cualquier ciudad y una accion equivale a decidir a que ciudad irse. La funcion de transicion refleja el nuevo estado `s'` y el pago que `r` que un agente recibiria tras haber decidido una accion `a` estando en el estado `s`. Tanto `s'` como `r` pueden ser aleatorios. Veamos como seria una instanciacion para nuestro problema.

In [7]:
state_space = cities
trans_fun(state, action) = action, - distArray[state, action] # la forma mas sencilla de definir funciones
action_set(state) = filter(x -> x ≠ state, cities)
europe_tour = RLEnv(state_space, trans_fun, action_set)

RLEnv(Any["Barcelona","Belgrade","Berlin","Brussels","Bucharest","Budapest","Copenhagen","Dublin","Hamburg","Istanbul"  …  "Moscow","Munich","Paris","Prague","Rome","Saint Petersburg","Sofia","Stockholm","Vienna","Warsaw"],trans_fun,action_set)

Por ejemplo, supongamos que actualmente estamos en Vienna y quiseramos ir a Warsaw. Entonces para europe tour la *accion* ir a Vienna dado que estamos en el *estado* Barcelona esta data por

In [8]:
new_state, reward = europe_tour.trans_fun("Barcelona", "Paris")
println("Partiendo de Barcelona, se llego a la ciudad $new_state obteniendo un pago de $reward")

Partiendo de Barcelona, se llego a la ciudad Paris obteniendo un pago de -831.59


## Agentes que aprenden

Para poder diseñar un agente necesitamos conocer el estado de un agente, la politica con la que toma decisiones y el pago total acumulado que ha recibido. Un agente solo esta definido dentro de una tarea de aprendizaje.

In [9]:
type RLAgent
    policy::Function # state -> action
    state::Any # su estado actual
    total_reward::Real # el reward que ha acumulado haste el momento
end

Para que quede mas claro, vamos a crear un agente que en el problema del tour de europa se mueve al azar a cualquier otra ciudad partiendo de su estado actual. Vamos a suponer que el estado inicial es barcelona.

In [10]:
policy(state) = rand(europe_tour.action_set(state)) # el agente mas tonto escoge una ciudad al azar sin depender del estado actual
state = "Barcelona"
total_reward = 0.0 # empezamos con cero reward acumulado
tour_agent = RLAgent(policy, state, total_reward)

RLAgent(policy,"Barcelona",0.0)

In [11]:
println("""La politica (aleatoria) que seguiria el agente tour_agent dado que actualmente se encuentra en
$(tour_agent.state) es ir a $(tour_agent.policy(tour_agent.state))""") # repetir este comando da distintos resultados

La politica (aleatoria) que seguiria el agente tour_agent dado que actualmente se encuentra en
Barcelona es ir a Bucharest


### Diseñando la interaccion

Para poder programar la interaccion entre un agente y su ambiente necesitamos programar *metodos*

Cuando un agente interactua con el ambiente deben ocurrir las siguientes cosas:

1. El agente escoge action data state segun su politica
2. El ambiente transiciona aleatoriamente, generando un nuevo estado new_state al gente y dando un reward en el proceso

Los metodos en julia son simplemente funciones cuyos argumentos son del tipo apropiado. A continuacion diseñamos un metodo de interaccion:

In [12]:
function interact!(agent::RLAgent, env::RLEnv)
    # la convencion (opcional) de julia es incluir '!' al final de una funcion si modifica sus argumentos 
    new_state, reward = env.trans_fun(agent.state, agent.policy(agent.state))
    agent.total_reward += reward
    agent.state = new_state
    return new_state, reward # a veces es conveniente regresar los rewards de cada iteracion
end
function reset!(agent::RLAgent, state = agent.state) # el estado es opcional
    agent.total_reward = 0
    agent.state = state    
end

reset! (generic function with 2 methods)

Para probar simulemos 20 interacciones

In [13]:
reset!(tour_agent, "Barcelona")
println("Agente inicia en $(tour_agent.state) con reward total $(tour_agent.total_reward)")
for i in 1:20
    interact!(tour_agent, europe_tour)
    println("Agente se mueve a $(tour_agent.state) con reward total $(tour_agent.total_reward)")
end

Agente inicia en Barcelona con reward total 0
Agente se mueve a Paris con reward total -831.59
Agente se mueve a Hamburg con reward total -1576.22
Agente se mueve a Dublin con reward total -2649.58
Agente se mueve a Paris con reward total -3426.41
Agente se mueve a Berlin con reward total -4303.37
Agente se mueve a Milan con reward total -5144.09
Agente se mueve a Belgrade con reward total -6029.41
Agente se mueve a Berlin con reward total -7028.66
Agente se mueve a Dublin con reward total -8343.82
Agente se mueve a Belgrade con reward total -10489.21
Agente se mueve a Dublin con reward total -12634.599999999999
Agente se mueve a Moscow con reward total -15427.009999999998
Agente se mueve a Madrid con reward total -18864.71
Agente se mueve a London con reward total -20128.079999999998
Agente se mueve a Barcelona con reward total -21265.75
Agente se mueve a Rome con reward total -22122.44
Agente se mueve a Kiev con reward total -23796.18
Agente se mueve a Brussels con reward total -2563

### Episodios

Un episodio termina cuando el agente visita todos los estados. Podemos programar una *Funcion* que simule un episodio usando nuestras funciones.

In [14]:
function visit_all!(agent::RLAgent, env::RLEnv)
    state_hist = [agent.state]
    reward_hist = [0.0]
    visited = Dict([(x, false) for x in env.state_space]) 
    while false in values(visited)
        new_state, reward = interact!(agent, env)
        push!(state_hist, new_state)
        push!(reward_hist, reward) 
        visited[new_state] = true
    end
    state_hist, reward_hist
end

visit_all! (generic function with 1 method)

In [15]:
reset!(tour_agent, "Barcelona")
state_hist, reward_hist = visit_all!(tour_agent, europe_tour)
println("Finished in $(size(state_hist, 1)) steps with total reward $(tour_agent.total_reward)")

Finished in 77 steps with total reward -93066.96999999999


In [16]:
# Policy evaluation
function policy_eval(agent::RLAgent, env::RLEnv, sweeps)
    state_space = env.state_space
    value = Dict([(x, 0.0) for x in state_space])
    for state in state_space, iter in 1:sweeps
        reset!(agent, state)
        visit_all!(agent, env)
        value[state] += (agent.total_reward - value[state]) / iter
    end
    return value
end


policy_eval (generic function with 1 method)

In [17]:
state_vals = policy_eval(tour_agent, europe_tour, 30)

Dict{SubString{String},Float64} with 24 entries:
  "Budapest"         => -1.1891e5
  "Warsaw"           => -1.11336e5
  "Rome"             => -1.17975e5
  "Munich"           => -1.18742e5
  "Kiev"             => -1.0652e5
  "Prague"           => -1.05644e5
  "Berlin"           => -107950.0
  "Istanbul"         => -124779.0
  "Madrid"           => -1.06871e5
  "Barcelona"        => -101311.0
  "Moscow"           => -1.16186e5
  "Dublin"           => -1.15444e5
  "Milan"            => -113154.0
  "Sofia"            => -1.09727e5
  "Vienna"           => -1.17511e5
  "Brussels"         => -1.09154e5
  "London"           => -1.14859e5
  "Saint Petersburg" => -1.11547e5
  "Copenhagen"       => -1.10915e5
  "Paris"            => -1.17799e5
  "Stockholm"        => -1.20112e5
  "Belgrade"         => -1.09022e5
  "Bucharest"        => -1.02303e5
  "Hamburg"          => -99511.5

In [18]:
# Policy improvement
new_policy = function(state) 
    # En este ejemplo: q(state,a) = distance(state, a) + v(a) 
    # Esgemos la politica greedy pi*(state) = argmax_a q(state, a)
    qx = [state_vals[state] - distArray[state, a] for a in cities if a ≠ state]
    maxval, pos = findmax(qx)
    return cities[pos]
end

(::#9) (generic function with 1 method)

In [19]:
new_policy("Paris")

"Brussels"